In [ ]:
"""
sales data from https://www.gov.uk/government/statistical-data-sets/price-paid-data-downloads#using-or-publishing-our-price-paid-data
"""

import json
import os
import re
from importlib import reload

import pandas as pd
import plotly.express as px
import requests
from dotenv import load_dotenv
from sqlalchemy import create_engine

import property_price_model.postcodes as pcodes

In [92]:
top_dir = os.getenv("TOP_DIR")

os.chdir(top_dir)

engine = create_engine("sqlite:///app.db")
conn = engine.raw_connection()

TEST_POSTCODE = "HP23"

sales = pd.read_sql(f"SELECT * FROM SALE where INCODE = '{TEST_POSTCODE}'", con=conn)
sales[["postcode", "paon", "saon", "street"]] = sales[["postcode", "paon", "saon", "street"]].fillna("")

epc = pcodes.get_epc_data(TEST_POSTCODE)

In [93]:
reload(pcodes)

sales["key"] = pcodes.add_land_reg_key(sales)
epc["key"] = pcodes.add_combined_epc_key(epc, sales["key"])

merged = pd.merge(sales, epc, on="key", how="left")

print(f"{sum(pd.notnull(merged.address1))} / {merged.shape[0]}")

failures = merged.loc[pd.isnull(merged["address1"])]["postcode_x"].values

failures[:10]

535 / 601


array(['HP23 5ED', 'HP23 5DR', 'HP23 4FD', 'HP23 5RX', 'HP23 5AX',
       'HP23 4FA', 'HP23 4FA', 'HP23 4BB', 'HP23 5AH', 'HP23 5HN'],
      dtype=object)

In [73]:
sales["key"].isin(epc[["key1", "key2"]].values.reshape(len(epc) * 2)).sum()

368

In [82]:
PCODE = failures[8]

print(sales.loc[sales["postcode"] == PCODE, "key"]) # ["postcode", "paon", "saon", "street", "joiner"]])
print(epc.loc[epc["postcode"] == PCODE, "key"]) # ["address1", "address2", "postcode", "joiner"]])


20     321 hill house 17 highgate hill n19 5na
21     319 hill house 17 highgate hill n19 5na
55     115 hill house 17 highgate hill n19 5na
82     108 hill house 17 highgate hill n19 5na
90     318 hill house 17 highgate hill n19 5na
91     221 hill house 17 highgate hill n19 5na
92     311 hill house 17 highgate hill n19 5na
104        hill house 17 highgate hill n19 5na
105    114 hill house 17 highgate hill n19 5na
143    326 hill house 17 highgate hill n19 5na
160    106 hill house 17 highgate hill n19 5na
165    307 hill house 17 highgate hill n19 5na
167    219 hill house 17 highgate hill n19 5na
180    117 hill house 17 highgate hill n19 5na
224    323 hill house 17 highgate hill n19 5na
225    109 hill house 17 highgate hill n19 5na
226    212 hill house 17 highgate hill n19 5na
227    403 hill house 17 highgate hill n19 5na
228    105 hill house 17 highgate hill n19 5na
229    104 hill house 17 highgate hill n19 5na
230    107 hill house 17 highgate hill n19 5na
231    312 hi

In [81]:
print(sales.loc[sales["postcode"] == PCODE, ["postcode", "paon", "saon", "street", "key"]])

print(epc.loc[epc["postcode"] == PCODE, ["address", "address2", "postcode", "key"]])

    postcode paon               saon            street  \
19   N19 4QN   20  GROUND FLOOR FLAT  SHAFTESBURY ROAD   
54   N19 4QN    8                     SHAFTESBURY ROAD   
64   N19 4QN   34                     SHAFTESBURY ROAD   
370  N19 4QN   84                     SHAFTESBURY ROAD   

                                               key  
19   ground floor flat 20 shaftesbury road n19 4qn  
54                      8 shaftesbury road n19 4qn  
64                     34 shaftesbury road n19 4qn  
370                    84 shaftesbury road n19 4qn  
                                     address             address2 postcode  \
220                     40, Shaftesbury Road                       N19 4QN   
372         Ground Floor, 26 Shaftsbury Road   26 Shaftsbury Road  N19 4QN   
989                     20b Shaftesbury Road                       N19 4QN   
1111                    70, Shaftesbury Road                       N19 4QN   
2295                    34, Shaftesbury Road          

In [94]:
merged["total-floor-area"] = pd.to_numeric(merged["total-floor-area"])
merged["number-heated-rooms"] = pd.to_numeric(merged["number-heated-rooms"])

merged[["price", "total-floor-area", "number-heated-rooms"]].describe()


,price,total-floor-area,number-heated-rooms
count,6.010000e+02,535.000000,500.00000
mean,5.189664e+05,100.705006,4.80200
std,2.923384e+05,50.966399,1.91994
min,5.000000e+02,31.000000,1.00000
25%,3.350000e+05,67.000000,4.00000
50%,4.485000e+05,89.000000,5.00000
75%,6.400000e+05,120.570000,6.00000
max,2.400000e+06,548.000000,22.00000


In [95]:
plot = merged[pd.notnull(merged["number-heated-rooms"])]

fig = px.scatter(merged, x="price", y="total-floor-area", color="number-heated-rooms", trendline = "ols")

fig.show()

In [96]:
fig.write_html("HP23.html")